In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torchvision
from tqdm import tqdm

import pickle
from matplotlib import pyplot as plt

## NES

In [2]:
from Week567_General_Code_Question import LeNet5, nes, evaluate
from Week567_General_Code_Question import visualize_adv

### 目标模型加载

In [3]:
mlp = LeNet5()
### 请将Week5保存的lenet5.pt迁移到/model路径下
mlp.load_state_dict(torch.load('model/lenet5.pt'))
mlp.eval()

criterion = nn.CrossEntropyLoss()

### 读入待攻击的样本

In [4]:
with open('data/Week567_img_label.pkl', 'rb') as f:
    data = pickle.load(f)
    imgs, labels = data['img'], data['label']

### 实现基于NES梯度估计的攻击

- 请在Week567_General_Code_Question.py的`nes(imgs, epsilon, model, labels, sigma, n)`函数中实现NES黑盒梯度估计
  - 核心思想：根据梯度定义，有$\nabla_x f(x) = \frac{f(x + \sigma) - f(x - \sigma)}{2\sigma}$；
  - 给定模型$P$，单个样本$x \in \mathbb{R}^{N}$，正确标签$y$，采样次数$n$，搜索方差$\sigma$，梯度$\nabla_x P(y|x)$可以通过如下方法估计：
    1. 采样噪声$u_i \gets \mathcal{N}(0_N, I_{N \cdot N})$
    2. 估计梯度$g_i \gets \frac{P(y|x + \sigma \cdot u_i) - P(x - \sigma \cdot u_i)}{2 \sigma}$
    3. $n$次采样求均值，$\nabla_x P(y|x) \approx \frac{1}{n} \sum_{i=1}^{n} g_i$

- 建议使用的API：
  - torch.randn [Link](https://pytorch.org/docs/stable/generated/torch.randn.html)
    - 采样随机高斯噪声

In [5]:
epsilon = 0.2
sigma = 0.001
n = 50
adv_xs = nes(imgs, epsilon, mlp, labels, sigma, n)

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [20, 784]

### 评测攻击效果

- 评估模型mlp对样本adv_xs的预测结果与真实标签labels的匹配率。该匹配率越低，则攻击效果越好。

In [ ]:
pred_label = evaluate(adv_xs, labels, mlp)

### 对抗样本可视化

In [ ]:
adv_imgs = adv_xs.reshape_as(imgs)
visualize_adv(adv_imgs, labels, pred_label)